In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

#### S1：数据信息和预处理：请概述所使用数据集的基本信息，并处理缺失值和异常值。
基本信息见实验报告。


In [9]:
#获取数据集
file_path = 'subdata.csv'
df = pd.read_csv(file_path, index_col = 0)
#处理缺失值
df_drop = df.dropna()
#处理异常值
cols = df_drop.columns[6:]
for c in cols:
    mu = df_drop[c].mean()
    sigma = df_drop[c].std()
    min = mu - 3 * sigma
    max = mu + 3 * sigma
    df_drop[c] = df_drop[c].mask(df_drop[c] < min, min)
    df_drop[c] = df_drop[c].mask(df_drop[c] > max, max)
#print(df_drop)

#处理字符串类型的数据
def STRATUM_str_to_num(s):
    return int(s[3:])

def PRIVATESCH_str_to_num(s):
    if s == 'private':
        return 1
    elif s == 'public':
        return -1
    else:
        return 0
    
df_drop['STRATUM'] = df_drop['STRATUM'].apply(STRATUM_str_to_num)
df_drop['PRIVATESCH'] = df_drop['PRIVATESCH'].apply(PRIVATESCH_str_to_num)
print(df_drop)

      CNTSCHID  Region  STRATUM  LANGTEST  PRIVATESCH  SCHLTYPE  STRATIO  \
2     72400003   72409     9018     156.0           1       1.0  10.4452   
3     72400004   72406      611     156.0          -1       3.0  12.6935   
4     72400005   72417     9038     156.0          -1       3.0   9.5074   
8     72400009   72416     9034     156.0          -1       3.0   7.4228   
9     72400010   72413     9028     156.0           1       2.0  16.1212   
...        ...     ...      ...       ...         ...       ...      ...   
1076  72401090   72414     9030     156.0          -1       3.0  12.6395   
1078  72401092   72407     9014     156.0           1       2.0  15.4943   
1079  72401093   72409      918     156.0           1       2.0  15.8450   
1081  72401095   72412     1223     156.0          -1       3.0  10.8000   
1087  72401101   72413     9025     156.0          -1       3.0  13.5645   

      SCHSIZE  RATCMP1  RATCMP2  ...  CLSIZE  CREACTIV  EDUSHORT  STAFFSHORT  \
2      

C:\Users\常征\AppData\Local\Temp\ipykernel_19816\189311765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[c] = df_drop[c].mask(df_drop[c] < min, min)
C:\Users\常征\AppData\Local\Temp\ipykernel_19816\189311765.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[c] = df_drop[c].mask(df_drop[c] > max, max)
C:\Users\常征\AppData\Local\Temp\ipykernel_19816\189311765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

#### S2：数据集划分(使用k折交叉验证法)：


In [ ]:
# 1分离特征和目标变量
X = df_drop.drop(columns=['TEACHBEHA'])  # 特征
y = df_drop['TEACHBEHA']  # 目标变量

# 2划分数据集
k = 5
kf = KFold(n_splits=k, shuffle=True)


平均MSE: 0.5761039349131776
平均R²: 0.3478928640910757


In [ ]:
# 2. 初始化模型
model1 = RandomForestRegressor()

# 3. 存储评估结果
mse_scores = []
r2_scores = []

# 4. 开始K折交叉验证
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 预测并评估
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    mse_scores.append(mse)
    r2_scores.append(r2)

# 5. 输出结果
print(f"平均MSE: {sum(mse_scores)/k}")
print(f"平均R²: {sum(r2_scores)/k}")

### 2.（60%）基于预处理后的数据集，编写算法代码进行频繁项集挖掘。

#### Q1.（30%）请参考以下 Apriori 产⽣频繁项集的算法流程，⾃⾏编写相应代码，分别以最⼩⽀持度阈值为0.25和0.5，挖掘频繁项集。


In [4]:
def Apriori(data, min_support=0.5):
    result = {}
    col_num = len(data.columns)
    val_dict = {}
    #找到各列可能取值
    for col in data.columns:
        val_dict[col] = data[col].unique().tolist()
    #计算频繁1项集
    r1 = {}
    for col in data.columns:
        r1[col] = {}
        for v in val_dict[col]:
            freq = len(data[data[col] == v])/len(data)
            if freq >= min_support:
                r1[col][(v,)] = (freq,)
        result[1] = r1
    #计算频繁k项集
    for i in range(2, col_num+1):
        if (i-1 not in result or  len(result[i-1]) == 1):
            break
        r = {}
        for col1 in result[i-1].keys():
            for col2 in result[i-1].keys():
                #计算候选k项集
                if col1[:-1] == col2[:-1] and col1[-1] < col2[-1]:
                    new_col = col1+col2[-1]
                    #用于存储可能的取值
                    test_val = []
                    for v1 in result[i-1][col1].keys():
                        for v2 in result[i-1][col2].keys():
                            if v1[:-1] == v2[:-1]:
                                new_val = (v1)+(v2[-1],)
                                test_val.append(new_val)
                    if len(test_val) == 0:
                        continue
                    #计算实际的频率
                    actual_val = []
                    actual_freq = []
                    for v in test_val:
                        a = data.copy()
                        for j in range(len(new_col)):
                            a = a[a[new_col[j]]==v[j]]
                        freq = len(a)/len(data)
                        if freq >= min_support:
                            actual_val.append(v)
                            actual_freq.append(freq)
                    r[new_col] = {tuple(actual_val[i]): (actual_freq[i],) for i in range(len(actual_val))}
        if r:   
            result[i] = r
    return result
print(Apriori(data_replace, 0.25))
print(Apriori(data_replace, 0.5))
        

{1: {'1': {(1.0,): (0.26816885063584933,), (2.0,): (0.3985784052995503,)}, '2': {(6.0,): (0.4188385474590204,), (7.0,): (0.3350901793917122,)}, '3': {(9.0,): (0.3363473719839466,), (10.0,): (0.3700014506068372,)}, '4': {(13.0,): (0.29447318795029254,), (14.0,): (0.4361491223828635,)}, '5': {(17.0,): (0.27334268168850634,), (18.0,): (0.48063439872346597,)}}, 2: {'12': {(2.0, 6.0): (0.2602872201537643,)}, '13': {(2.0, 10.0): (0.2744064600357816,)}, '14': {(2.0, 14.0): (0.26183453411343743,)}, '15': {(2.0, 18.0): (0.27401963154586334,)}, '23': {}, '24': {(6.0, 14.0): (0.2694260432280837,)}, '25': {(6.0, 18.0): (0.2733910352497461,)}, '34': {(10.0, 14.0): (0.27450316715826123,)}, '35': {(10.0, 18.0): (0.2713118321164354,)}, '45': {(14.0, 18.0): (0.33054494463517237,)}}, 3: {'123': {}, '124': {}, '125': {}, '134': {}, '135': {}, '145': {}, '245': {}, '345': {}}}
{1: {'1': {}, '2': {}, '3': {}, '4': {}, '5': {}}}


#### Q2.（15%）当最小支持度为0.5时，频繁项集数量较少。请将各特征原始取值为1和2的单元格统一修改其值为0，取值为3和4的单元格统一修改其值为1。重复T1-Q2的项集索引构建过程，并以最小支持度阈值为0.5，挖掘频繁项集。

In [5]:
data_rereplace = data_drop.copy()
for col in data_rereplace.columns:
    data_rereplace[col]= data_rereplace[col].replace(1, 0)
    data_rereplace[col]= data_rereplace[col].replace(2, 0)
    data_rereplace[col]= data_rereplace[col].replace(3, 1)
    data_rereplace[col]= data_rereplace[col].replace(4, 1)
print('更新前'+'-'*20)
print(data_rereplace)

for i in rename:
    for j in range(0, 2):
        ind2val[crt_idx] = f'[{i}]=[{j}]'
        crt_idx += 1

i = 0
for col in data_drop.columns:
    for value in data_drop[col].unique():
        data_rereplace[col]= data_rereplace[col].replace(value, value+i)
    i+=2
print('更新后'+'-'*20)
print(data_rereplace)
print('频繁项集'+'-'*20)
print(Apriori(data_rereplace, 0.5))

更新前--------------------
         1    2    3    4    5
1      0.0  0.0  0.0  0.0  0.0
2      0.0  0.0  0.0  0.0  0.0
3      0.0  1.0  0.0  0.0  0.0
4      0.0  1.0  0.0  0.0  0.0
5      0.0  1.0  0.0  0.0  0.0
...    ...  ...  ...  ...  ...
21899  0.0  1.0  0.0  0.0  0.0
21900  1.0  1.0  0.0  0.0  0.0
21901  1.0  1.0  1.0  0.0  0.0
21902  1.0  1.0  1.0  1.0  1.0
21903  1.0  1.0  0.0  0.0  0.0

[20681 rows x 5 columns]
更新后--------------------
         1    2    3    4    5
1      0.0  0.0  0.0  0.0  0.0
2      0.0  0.0  0.0  0.0  0.0
3      0.0  3.0  0.0  0.0  0.0
4      0.0  3.0  0.0  0.0  0.0
5      0.0  3.0  0.0  0.0  0.0
...    ...  ...  ...  ...  ...
21899  0.0  3.0  0.0  0.0  0.0
21900  1.0  3.0  0.0  0.0  0.0
21901  1.0  3.0  5.0  0.0  0.0
21902  1.0  3.0  5.0  7.0  9.0
21903  1.0  3.0  0.0  0.0  0.0

[20681 rows x 5 columns]
频繁项集--------------------
{1: {'1': {(1.0,): (0.625211546830424,)}, '2': {(3.0,): (0.6224553938397563,)}, '3': {(5.0,): (0.547507373918089,)}, '4': {(7.0,): 

#### Q3.（15%）分析Q1和Q2的结果，你有什么发现？请根据各特征定义，分析产⽣这种情况的原因。
见实验报告

### 3.（25%）基于T2-Q2得到的频繁项集挖掘结果，编写算法代码进⾏关联规则提取。

#### Q1.（15%）请以最⼩置信度阈值为0.8，提取形如X->{1}的关联规则，并输出它们的置信度和提升度。

In [6]:
def find_rule(data, confidence=0.8):
    d1 = data[1]
    d2 = data[2]
    for col, val_freq in d2.items():
        for val, freq in val_freq.items():
            for i in range(len(col)):
                target = col[i]
                condition = col[:i] + col[i+1:]
                target_freq = d1[target][(val[i],)]
                condition_freq = d1[condition][(val[(i+1)%2],)]
                print(f'关联规则: {val[i]} -> {val[(i+1)%2]},置信度: {freq[0]/target_freq[0]}, 提升度: {freq[0]/(target_freq[0]*condition_freq[0])}')
find_rule(Apriori(data_rereplace, 0.5))


关联规则: 1.0 -> 3.0,置信度: 0.8226604795050271, 提升度: 1.3216376428682872
关联规则: 3.0 -> 1.0,置信度: 0.8263031150469975, 提升度: 1.3216376428682872
关联规则: 1.0 -> 5.0,置信度: 0.8125290023201858, 提升度: 1.4840512494024343
关联规则: 5.0 -> 1.0,置信度: 0.9278459772145193, 提升度: 1.4840512494024343
关联规则: 1.0 -> 7.0,置信度: 0.8010054137664347, 提升度: 1.3423217698811796
关联规则: 7.0 -> 1.0,置信度: 0.8392350700915647, 提升度: 1.3423217698811796
关联规则: 1.0 -> 9.0,置信度: 0.8194895591647333, 提升度: 1.2957082242420377
关联规则: 9.0 -> 1.0,置信度: 0.8100917431192662, 提升度: 1.2957082242420377
关联规则: 7.0 -> 9.0,置信度: 0.8674337573940524, 提升度: 1.3715135731396328
关联规则: 9.0 -> 7.0,置信度: 0.8184250764525994, 提升度: 1.3715135731396328


#### Q2.（10%）参考项集索引的对应关系，对以上频繁项集和关联规则结果进⾏简要分析和总结。
见实验报告